<a href="https://colab.research.google.com/github/Aryan-Kamboj11/Multimodal-Hate-Speech-Detection/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import kagglehub

path = kagglehub.dataset_download("victorcallejasf/multimodal-hate-speech")

100%|██████████| 5.97G/5.97G [01:21<00:00, 78.6MB/s]

Extracting files...


In [5]:
print(f"Path to dataset is {path}")

Path to dataset is /root/.cache/kagglehub/datasets/victorcallejasf/multimodal-hate-speech/versions/1


In [6]:
import os
os.listdir(path)

['MMHS150K_GT.json',
 'img_resized',
 'splits',
 'img_txt',
 'hatespeech_keywords.txt',
 'MMHS150K_readme.txt']

In [7]:
import pandas as pd

data = pd.read_json(path+"/MMHS150K_GT.json",orient='index')
print(data.shape)


(149823, 5)


In [8]:
print(data.head())

                                                                         img_url  \
2005-04-28 09:09:13.714016256  http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...   
2003-09-08 11:20:48.816660480  http://pbs.twimg.com/ext_tw_video_thumb/106301...   
2005-02-20 19:22:48.075374593     http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg   
2005-04-26 23:35:34.635618305  http://pbs.twimg.com/ext_tw_video_thumb/111401...   
2002-10-22 02:08:00.215592966     http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg   

                                  labels  \
2005-04-28 09:09:13.714016256  [4, 1, 3]   
2003-09-08 11:20:48.816660480  [5, 5, 5]   
2005-02-20 19:22:48.075374593  [0, 0, 0]   
2005-04-26 23:35:34.635618305  [1, 0, 0]   
2002-10-22 02:08:00.215592966  [1, 0, 1]   

                                                                       tweet_url  \
2005-04-28 09:09:13.714016256  https://twitter.com/user/status/11146793537140...   
2003-09-08 11:20:48.816660480  https://twitter.com/user/status

In [9]:
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np

In [18]:
def load_dataset(data_path,output_dir="Dataset",num_samples = 2000):
  # os.mkdir(output_dir)
  # os.mkdir(os.path.join(output_dir,"images"))
  df = pd.read_json(data_path,orient='index')

  valid_text = []
  valid_images_path = []
  valid_labels = []

  img_cnt = 0
  idx = 0
  while(len(valid_text)<num_samples and idx < len(df)):
    row = df.iloc[idx]
    try:
      image_url = row["img_url"]
      image_name = f"{img_cnt}.jpg"
      img_path = os.path.join(output_dir,"images",image_name)
      if os.path.exists(img_path):
        print(f"Image {image_name} already exists")
        valid_text.append(row["tweet_text"])
        valid_labels.append(row["labels"]) # Changed 'label' to 'labels' based on data variable
        valid_images_path.append(img_path)
        img_cnt+=1
      else:
        response = requests.get(image_url,timeout = 10)
        if response.status_code == 200 :
          img = Image.open(BytesIO(response.content))
          img.save(img_path)
          valid_text.append(row["tweet_text"])
          valid_labels.append(row["labels"]) # Changed 'label' to 'labels' based on data variable
          valid_images_path.append(img_path)
          img_cnt+=1
        else:
          print(f"failed to download image {image_url}")
          # Do not increment img_cnt or append to lists if download fails
    except Exception as e:
      print(e)
      # Do not increment img_cnt or append to lists if any other exception occurs
    idx+=1 # Always increment idx to move to the next row

  valid_df = pd.DataFrame({"text":valid_text,"label":valid_labels,"image_path":valid_images_path})
  valid_df.to_csv(os.path.join(output_dir,"valid.csv"),index=False)
  return valid_df

In [19]:
data_path = path+"/MMHS150K_GT.json"

dataset_df = load_dataset(data_path,output_dir="Dataset",num_samples = 2000)

print(dataset_df.head())

Image 0.jpg already exists
failed to download image http://pbs.twimg.com/ext_tw_video_thumb/1063019643709747200/pu/img/wK5HgoX6tFfxWJmi.jpg
failed to download image http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg
failed to download image http://pbs.twimg.com/media/D1zG0qnX4AAw9SC.jpg
failed to download image http://pbs.twimg.com/media/D3fQcCCWAAIG8tO.jpg
failed to download image http://pbs.twimg.com/tweet_video_thumb/D39S8tbW4AIflmW.jpg
failed to download image http://pbs.twimg.com/tweet_video_thumb/D345PrBX4AABczg.jpg
failed to download image http://pbs.twimg.com/media/D3QOSgnWkAEvioq.jpg
failed to download image http://pbs.twimg.com/ext_tw_video_thumb/1107795597002788866/pu/img/pPHyr5ms0F8Pv4D4.jpg
failed to download image http://pbs.twimg.com/tweet_video_thumb/D1tcqNcWsAAkNaY.jpg
failed to download image http://pbs.twimg.com/media/D2C80RkX0AIw0Fy.jpg
failed to download image http://pbs.twimg.com/media/DqS9h79XQAE4BVI.jpg
failed to download image http://pbs.twimg.com/ext_tw_video_thum

In [20]:
len(dataset_df)

2000